In [11]:
# Homework 4
# By: Group 8

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from pandas.tseries.holiday import USFederalHolidayCalendar
import datetime as dt
from sklearn.pipeline import make_pipeline

counts = pd.read_csv('data/FremontBridge.csv', index_col='Date', parse_dates=True)
weather = pd.read_csv('data/BicycleWeather.csv', index_col='DATE', parse_dates=True)

# ||________________________________________________This code is used to create the array holding relavent data
# Constructing the daily array that holds the data 
daily = counts.resample('d').sum()
daily['Total'] = daily.sum(axis=1)
daily = daily[['Total']] # remove other columns
# add an indicator about Mon - Sun
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(7):
    daily[days[i]] = (daily.index.dayofweek == i).astype(float)
# add an indicator about holiday
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', '2016')
daily = daily.join(pd.Series(1, index=holidays, name='holiday'))
# replace missing data with 0
daily['holiday'].fillna(0, inplace=True)
def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """Compute the hours of daylight for the given date"""
    days = (date - dt.datetime(2000, 12, 21)).days
    m = (1. - np.tan(np.radians(latitude))
         * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
    return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.
daily['daylight_hrs'] = list(map(hours_of_daylight, daily.index))
# temperatures are in 1/10 deg C; convert to C
weather['TMIN'] /= 10
weather['TMAX'] /= 10
weather['Temp (C)'] = 0.5 * (weather['TMIN'] + weather['TMAX'])
# precip is in 1/10 mm; convert to inches
weather['PRCP'] /= 254
weather['dry day'] = (weather['PRCP'] == 0).astype(int)
daily = daily.join(weather[['PRCP', 'Temp (C)', 'dry day']])
daily['annual'] = (daily.index - daily.index[0]).days / 365.
daily.dropna(axis=0, how='any', inplace=True)
column_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'holiday',
                'daylight_hrs', 'PRCP', 'dry day', 'Temp (C)', 'annual']
X = daily[column_names]
y = daily['Total']

#||__________________________________________Linear Regression
from sklearn.model_selection import cross_val_score

model = LinearRegression(fit_intercept=False)
model.fit(X, y)
print("Linear Regression:", sum(cross_val_score(model, X, y, cv=10))/10)

#||__________________________________________Ridge
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge
from scipy.stats import uniform

parameters = {'alpha': uniform()}
ridgeModel = Ridge()
randSearchCV = RandomizedSearchCV(ridgeModel, parameters, n_iter=100, cv=10, n_jobs=-1)
search = randSearchCV.fit(X,y)
print("Ridge score:", search.best_score_)
print("Ridge alpha:", search.best_estimator_.alpha)

#||__________________________________________Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Lasso
from scipy.stats import uniform

parameters = {'alpha': uniform()}
lassoModel = Lasso()
randSearchCV = RandomizedSearchCV(lassoModel, parameters, n_iter=100, cv=10, n_jobs=-1)
search = randSearchCV.fit(X,y)
print("Lasso score:", search.best_score_)
print("Lasso alpha:", search.best_estimator_.alpha)

Linear Regression: 0.7691752923789885
Ridge score: 0.7699746169255028
Ridge alpha: 0.9974662396744683
Lasso score: 0.76990360237112
Lasso alpha: 0.9981789827089937
